In [1]:
using LinearAlgebra
using JuMP
using Gurobi
using Plots
using Random
using LaTeXStrings

In [2]:
function project_hyperplane(A,x0,lcon,ucon)
    if A'*x0 > ucon
        return x0 - ((A'*x0-ucon)/(A'*A))*A
    elseif A'*x0<lcon
        return x0 - ((A'*x0-lcon)/(A'*A))*A
    elseif lcon<=A'*x0 && A'*x0<=ucon
        return x0
    end
end

project_hyperplane (generic function with 1 method)

In [3]:
function project_bound(x0,lb,ub)
    return min.(max.(x0,lb),ub)
end

project_bound (generic function with 1 method)

In [4]:
"""
Dykstra's Cyclic Projection Algorithm
- 目标: 找到点 `x` 在多个凸集合 C₁, C₂, ..., Cₖ 的交集上的投影
- 输入: 
  - x0: 初始点
  - sets: 凸集合的投影函数数组（每个函数接受向量，返回投影后的向量）
  - max_iter: 最大迭代次数
  - tol: 收敛容忍度
- 输出: 投影后的点 x 和修正项历史记录
"""
function dykstra_projection(x0::Vector{Float64},A,low_con,up_con,low_var,up_var,
        max_iter::Int = 1000, tol::Float64 = 1e-3
)
#    k = length(sets)
#    println(size(A,1))
    k = size(A,1)
    x = copy(x0)
    n = length(x)
    # 初始化修正项
    y = [zeros(n) for _ in 1:(k+1)]
    history = Float64[]
    
    for iter in 1:max_iter
        x_prev = copy(x)
        for i in 1:(k+1)
            # 计算修正后的投影
            x_temp = x - y[i]
            # 对当前约束进行投影
            x_proj = x;
            if i<=k
                #对low_con<=Ax<=up_con约束进行投影
                x_proj = project_hyperplane(A[i,:],x_temp,low_con[i],up_con[i])
            elseif i==k+1
                x_proj = project_bound(x_temp,low_var,up_var)
            end
            #x_proj = sets[i](x_temp)
            # 更新修正项
            y[i] = x_proj - (x_temp - y[i])
            # 更新当前估计
            x = x_proj
        end
        # 检查收敛性
        residual = norm(x - x_prev)
        push!(history, residual)
        if residual < tol
            #println("Converged at iteration $iter")
            break
        end
    end
   # return x, history
    return x
end

dykstra_projection

In [5]:

Random.seed!(223)  # 设置一个具体的种子值

N = 10;
ub = rand(100:500,N);
lb = rand(-300:99,N);

con_ub = rand(100:500,N);
con_lb = rand(-300:99,N); 
A = randn(N,N); 
#A = A'*A; A = (A + A')/2

c = rand(N);
Q = randn(N,N); Q = Q'*Q; Q = (Q + Q')/2

#= 对矩阵Q的特征根进行测试，目的是验证矩阵Q的正定性
λ=eigen(Q);

for k in range(1,length(λ.values))
    if λ.values[k] >= 0
        println(false)
    end
end
=#

model = Model(Gurobi.Optimizer)
set_silent(model)
@variable(model, x[1:N])
@constraint(model, x.<=ub)
@constraint(model, lb.<= x)

@constraint(model, A*x .<= con_ub)
@constraint(model, con_lb .<= A*x)

@objective(model, Min, dot(c,x) + 0.5*transpose(x)*Q*x)

optimize!(model)
#print(value.(x),"\n")
print(solution_summary(model),"\n")
println(objective_value(model))

opt_x = value.(x)

Set parameter LicenseID to value 2590635
* Solver : Gurobi

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Model was solved to optimality (subject to tolerances), and an optimal solution is available."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 7.36389e+04
  Objective bound    : 7.36389e+04
  Dual objective value : 7.36389e+04

* Work counters
  Solve time (sec)   : 1.79999e-02
  Simplex iterations : 0
  Barrier iterations : 8
  Node count         : 0

73638.90052140859


10-element Vector{Float64}:
  39.79764526249099
  42.000000000001144
 -57.999999999998785
  72.00000000000006
 -36.730856758513454
 -15.875671133488225
   5.616590240100351
  83.00000000000018
  63.000000000002544
 -61.94199060150051

In [6]:
var_num = N;
Max_Iters = 1000;
x0 = zeros(var_num,1);
xk = zeros(var_num,Max_Iters);
cost = zeros(1,Max_Iters);
cost[1,1] = 0.5*dot(Q*xk[:,1],xk[:,1]);

for k in range(1,Max_Iters-1)
    grad = Q*xk[:,k] + c;
    x = xk[:,k] - grad/(k+1);
    #xk[:,k+1] = CyclicDykstraAlgorithm(x,ConMatrix,BoundMatrix,2000);
    #xk[:,k+1] = CyclicDykstraAlgorithm(x,A,ucon,100);
     xk[:,k+1] = dykstra_projection(x,A,con_lb,con_ub,lb,ub)
    #xk[:,k+1] = dykstra_projection(x,A,con_lb,con_ub,lb,ub,1000,1/k)
    cost[1,k+1] = 0.5*dot(Q*xk[:,k+1],xk[:,k+1]) + dot(c,xk[:,k+1]);
    if mod(k,1000)==0
        print(k,"，")
    end
end

In [7]:
cost[1,end-1]

73639.14107783046

In [11]:
plot(1:900,abs.(cost[1,1:900].-objective_value(model)),yscale=:log10,linewidth=2,legend=false,color ="red")
xlabel!("Iteration k")
ylabel!(L"||f_{k}-f^{*}||")
savefig("ferror.pdf")

"C:\\Users\\pengl\\ferror.pdf"

a1 = zeros(1,2)
a2 = zeros(1,2)
abs.(a1[1,1:2].-a2[1,1:2])

In [12]:
x_error = zeros(Max_Iters);
for k in range(1,Max_Iters)
    x_error[k] = norm(xk[:,k]-opt_x)
end

#plot(1:Max_Iters,x_error[1:Max_Iters],yscale=:log10,linewidth = 2)
plot(1:900,x_error[1:900],yscale=:log10,linewidth = 2,legend=false)
xlabel!("Iteration k")
ylabel!(L"||x_{k}-x^{*}||")
savefig("xerror.pdf")

"C:\\Users\\pengl\\xerror.pdf"

In [13]:
norm(xk[:,end]-opt_x)

0.6888915923172326